In [82]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon

In [83]:
import json

tweets_file = open('geotagged_tweets_20160812-0912.1000.jsons', "r")

def flatten_tweets(tweets_json):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []
    
# Iterate through each tweet
    for tweet in tweets_json:
        tweet_obj = json.loads(tweet)            
        tweets_list.append(tweet_obj)
    return tweets_list

tweets = flatten_tweets(tweets_file)


In [103]:
import pandas as pd

from pandas.io.json import json_normalize
# Thank gawd for this package - all json fields straight into columns in a dataframe, no bullshit
tweets_df = json_normalize(tweets)

tweets_df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quoted_status.place.country,quoted_status.place.bounding_box.type,quoted_status.place.bounding_box.coordinates,quoted_status.quoted_status_id,quoted_status.quoted_status_id_str,geo.type,geo.coordinates,coordinates.type,coordinates.coordinates,quoted_status.scopes.followers
0,Sun Sep 11 02:05:23 +0000 2016,774790914589990912,774790914589990912,@HillaryClinton @POTUS @JudgeJeanine @NBA @NFL @MLB @UCLA @greta @dukeblueplanet There IS A PLAN U PIC ❤️✝👀 https://t.co/UDsJmesa5w,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",False,NaN,None,1.339836e+09,1339835893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wed Aug 17 20:18:54 +0000 2016,766006411994292224,766006411994292224,IF THEY ONLY HAD A BRAIN! \n#NeverTrump #BuhByeTRUMP \n#TrumpIsUnfit https://t.co/YjYABvQ8pw,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri Sep 02 23:06:23 +0000 2016,771846762533224448,771846762533224448,"Attorney's first line of defense to those they represent: just keep saying ""I don't recall"" ""I don't remember"" #lyingHillary @HillaryClinton","<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thu Sep 01 03:01:51 +0000 2016,771181247179558912,771181247179558912,@kikesma @norma22flores @EPN @realDonaldTrump No lo recibió como presidente,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",False,7.710818e+17,771081824512643073,8.541293e+07,85412926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sat Sep 10 05:22:45 +0000 2016,774478196502700032,774478196502700032,"Classy.. Better than I expected. But on a serious note, why would u ever want to go to an event like this? Crazy. https://t.co/jOdETLmkSn","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Tue Aug 16 01:40:52 +0000 2016,765362658346278913,765362658346278913,Esta bien guey @realDonaldTrump .,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,NaN,None,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,Sat Sep 10 16:06:30 +0000 2016,774640200932659200,774640200932659200,@realDonaldTrump @tonyschwartz https://t.co/ak6z0Lcb9R,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",False,7.744843e+17,774484342030602240,2.507388e+07,25073877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,Thu Aug 18 23:03:18 +0000 2016,766410172331728896,766410172331728896,@realDonaldTrump please remind people and MSM (which are not people)of Obama's shovel ready jobs promise. And him saying not so shovel ready,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",False,NaN,None,2.507388e+07,25073877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,Sun Aug 21 06:35:22 +0000 2016,767248713693605889,767248713693605889,@HillaryClinton how do you sleep at night?,"<a href=""http://twitter.com/download/android"" rel=""nofollow"">Twitter for Android</a>",False,7.671235e+17,767123506148630528,1.339836e+09,1339835893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
cols = []

for col in tweets_df.columns:
    if col in ('text', 'lang', 'user.followers_count', 'user.friends_count', 'user.listed_count', 'user.favourites_count', 'user.statuses_count'):
        cols.append(col)

neat_df = tweets_df[cols]
neat_df

,text,lang,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,user.statuses_count
0,@HillaryClinton @POTUS @JudgeJeanine @NBA @NFL @MLB @UCLA @greta @dukeblueplanet There IS A PLAN U PIC ❤️✝👀 https://t.co/UDsJmesa5w,en,28,32,3,3,4638
1,IF THEY ONLY HAD A BRAIN! \n#NeverTrump #BuhByeTRUMP \n#TrumpIsUnfit https://t.co/YjYABvQ8pw,en,10646,9586,321,50167,102678
2,"Attorney's first line of defense to those they represent: just keep saying ""I don't recall"" ""I don't remember"" #lyingHillary @HillaryClinton",en,4053,4341,35,158,12856
3,@kikesma @norma22flores @EPN @realDonaldTrump No lo recibió como presidente,es,927,488,14,63,51264
4,"Classy.. Better than I expected. But on a serious note, why would u ever want to go to an event like this? Crazy. https://t.co/jOdETLmkSn",en,167,83,60,28583,14469
...,...,...,...,...,...,...,...
995,Esta bien guey @realDonaldTrump .,es,2159,1206,38,26124,35054
996,@realDonaldTrump @tonyschwartz https://t.co/ak6z0Lcb9R,und,117,90,15,6684,8487
997,@realDonaldTrump please remind people and MSM (which are not people)of Obama's shovel ready jobs promise. And him saying not so shovel ready,en,338,452,7,4397,2414
998,@HillaryClinton how do you sleep at night?,en,165,867,2,475,1396


In [106]:
from textblob import TextBlob as tb

pd.set_option('display.max_colwidth', -1)
neat_df['calc_sentiment'] = neat_df['text'].apply(lambda x: tb(x).sentiment[0] )
neat_df[['text','calc_sentiment']]


C:\Python37\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,text,calc_sentiment
0,@HillaryClinton @POTUS @JudgeJeanine @NBA @NFL @MLB @UCLA @greta @dukeblueplanet There IS A PLAN U PIC ❤️✝👀 https://t.co/UDsJmesa5w,0.000000
1,IF THEY ONLY HAD A BRAIN! \n#NeverTrump #BuhByeTRUMP \n#TrumpIsUnfit https://t.co/YjYABvQ8pw,0.000000
2,"Attorney's first line of defense to those they represent: just keep saying ""I don't recall"" ""I don't remember"" #lyingHillary @HillaryClinton",0.250000
3,@kikesma @norma22flores @EPN @realDonaldTrump No lo recibió como presidente,0.000000
4,"Classy.. Better than I expected. But on a serious note, why would u ever want to go to an event like this? Crazy. https://t.co/jOdETLmkSn",-0.086667
...,...,...
995,Esta bien guey @realDonaldTrump .,0.000000
996,@realDonaldTrump @tonyschwartz https://t.co/ak6z0Lcb9R,0.000000
997,@realDonaldTrump please remind people and MSM (which are not people)of Obama's shovel ready jobs promise. And him saying not so shovel ready,0.200000
998,@HillaryClinton how do you sleep at night?,0.000000


In [107]:
print(neat_df.describe())

       user.followers_count  user.friends_count  user.listed_count  \
count  1000.000000           1000.000000         1000.000000         
mean   3109.432000           1966.535000         69.662000           
std    21531.864353          7968.453702         207.638097          
min    0.000000              0.000000            0.000000            
25%    94.000000             155.500000          4.000000            
50%    424.500000            580.500000          18.000000           
75%    1669.000000           1971.250000         54.000000           
max    541663.000000         190843.000000       3863.000000         

       user.favourites_count  user.statuses_count  calc_sentiment  
count  1000.000000            1000.000000          1000.000000     
mean   7483.658000            26064.585000         0.036832        
std    17888.113236           62147.288141         0.312130        
min    0.000000               3.000000            -1.000000        
25%    347.000000            

In [174]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [175]:
#Look at missing values
neat_df.isna().head()
print(neat_df.isna().sum())

lang                     0
user.followers_count     0
user.friends_count       0
user.listed_count        0
user.favourites_count    0
user.statuses_count      0
calc_sentiment           0
dtype: int64


In [60]:
#In case values are missing use following:
#neat_df.fillna(neat_df.mean(), inplace=True)

In [176]:
neat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
lang                     1000 non-null int64
user.followers_count     1000 non-null int64
user.friends_count       1000 non-null int64
user.listed_count        1000 non-null int64
user.favourites_count    1000 non-null int64
user.statuses_count      1000 non-null int64
calc_sentiment           1000 non-null float64
dtypes: float64(1), int64(6)
memory usage: 54.8 KB


In [177]:
del neat_df['text']
neat_df

KeyError: 'text'

In [178]:
neat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
lang                     1000 non-null int64
user.followers_count     1000 non-null int64
user.friends_count       1000 non-null int64
user.listed_count        1000 non-null int64
user.favourites_count    1000 non-null int64
user.statuses_count      1000 non-null int64
calc_sentiment           1000 non-null float64
dtypes: float64(1), int64(6)
memory usage: 54.8 KB


In [179]:
labelEncoder = LabelEncoder()
labelEncoder.fit(neat_df['lang'])
neat_df['lang'] = labelEncoder.transform(neat_df['lang'])

C:\Python37\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [180]:
neat_df

,lang,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,user.statuses_count,calc_sentiment
0,1,28,32,3,3,4638,0.000000
1,1,10646,9586,321,50167,102678,0.000000
2,1,4053,4341,35,158,12856,0.250000
3,2,927,488,14,63,51264,0.000000
4,1,167,83,60,28583,14469,-0.086667
...,...,...,...,...,...,...,...
995,2,2159,1206,38,26124,35054,0.000000
996,11,117,90,15,6684,8487,0.000000
997,1,338,452,7,4397,2414,0.200000
998,1,165,867,2,475,1396,0.000000


In [181]:
X = np.array(neat_df.drop(['calc_sentiment'], 1).astype(float))
Y = np.array(neat_df['calc_sentiment'])

In [182]:
neat_df

,lang,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,user.statuses_count,calc_sentiment
0,1,28,32,3,3,4638,0.000000
1,1,10646,9586,321,50167,102678,0.000000
2,1,4053,4341,35,158,12856,0.250000
3,2,927,488,14,63,51264,0.000000
4,1,167,83,60,28583,14469,-0.086667
...,...,...,...,...,...,...,...
995,2,2159,1206,38,26124,35054,0.000000
996,11,117,90,15,6684,8487,0.000000
997,1,338,452,7,4397,2414,0.200000
998,1,165,867,2,475,1396,0.000000


In [184]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [185]:
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [186]:
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == Y[i]:
        correct += 1

print(correct/len(X))

0.497
